In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [3]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variables(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成一个截断的正态分布
    return t.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层:x=[batch,in_height,in_width,in_channels] ,W=[filter_height,filter_width,in_channels,out_channels]
s
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层:2*2的窗口，步长为2
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#改变x的格式转为4D的向量[batch,in_height,in_width,in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variables([5,5,1,32])#5*5的采样窗口，32个卷积核从一个平面抽取特征
b_conv1 = bias_variable([32]) #每一个卷积核一个偏置

#把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1) #进行max_pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variables([5,5,32,64]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64]) #每一个卷积核一个偏置

#把h_pool1和权重向量进行卷积，再加上偏置值，然后应用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) #进行max_pooling

#28*28的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14,第二次池化后变为7*7
#进行上面操作后得到64张7*7的平面

#初始化第一个全连接层的权值
W_fc1 = weight_variables([7*7*64,1024]) #上一层有7*7*64个神经元，全连接层有1024个神经元
b_fc1 = bias_variable([1024]) 

#把池化层2的输出扁平化为1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fc1  = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

#keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层
W_fc2 = weight_variables([1024,10])
b_fc2 = bias_variable([10])

#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果放在一个布尔列表中
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))

        

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter 0, Testing Accuracy= 0.8626
Iter 1, Testing Accuracy= 0.9718
Iter 2, Testing Accuracy= 0.9791
Iter 3, Testing Accuracy= 0.9812
Iter 4, Testing Accuracy= 0.9829
Iter 5, Testing Accuracy= 0.9868
Iter 6, Testing Accuracy= 0.987
Iter 7, Testing Accuracy= 0.9879
Iter 8, Testing Accuracy= 0.9881
Iter 9, Testing Accuracy= 0.9896
Iter 10, Testing Accuracy= 0.99
Iter 11, Testing Accuracy= 0.9892
Iter 12, Testing Accuracy= 0.9897
Iter 13, Testing Accuracy= 0.9897
Iter 14, Testing Accuracy= 0.9895
Iter 15, Testing Accuracy= 0.9897
Iter 16, Testing Accuracy= 0.9907
Iter 17, Testing Accuracy= 0.9908
Iter 18, Testing A